# This Notebook allows analyzing the computed diffusion Coefficients as well as the `Q` matrices

In [ ]:
import sys
import re

import numpy as np
import pandas as pd
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt

# Load plotting style
sys.path.append('/home/tobiac/polybox/studium/mscThesis/personal_repo/data_visualization/src')
from mscPlotting import set_plotting_environment, SizeParams
import diffusionCoeffTools as Dtools 
set_plotting_environment()

# Load plot post-processors
from mscPlotting import prepare_emittance_plots, prepare_imshow_plots

# Lineplot Kwargs specific to the first section of the Notebook
lineplot_kwargs_dict = {
                   'logx' : True, 
                   'logy' : True, 
                   'color' : None}

# Where to store figures
figure_dir = '/home/tobiac/polybox/studium/mscThesis/personal_repo/report/figures/tmp/'

# Langevin Data from DIH (r256, v64)

## Load $D$ & $Q$ Field values

In [ ]:
coeffs_dict = {"D" : pd.read_csv("spectralHessian_hockney/Dfield_it1199.csv"),
               "Q" : pd.read_csv("spectralHessian_hockney/Qfield_it1199.csv"),
              "Fd" : pd.read_csv("spectralHessian_hockney/FdNorm_it1199.csv")}

# coeffs_dict = {"D" : pd.read_csv("spectralHessian_vico/Dfield_it0200.csv"),
#                "Q" : pd.read_csv("spectralHessian_vico/Qfield_it1199.csv"),
#               "Fd" : pd.read_csv("spectralHessian_vico/FdNorm_it1199.csv")}

# coeffs_dict = {"D" : pd.read_csv("FDhessian_vico/Dfield_it0200.csv"),
#                "Q" : pd.read_csv("FDhessian_vico/Qfield_it1199.csv"),
#               "Fd" : pd.read_csv("FDhessian_vico/FdNorm_it1199.csv")}

# coeffs_dict = {"D" : pd.read_csv("spectralHessian_vico_noVelspaceNorm/Dfield_it1199.csv"),
#                "Q" : pd.read_csv("spectralHessian_vico_noVelspaceNorm/Qfield_it1199.csv"),
#               "Fd" : pd.read_csv("spectralHessian_vico_noVelspaceNorm/FdNorm_it1199.csv")}

# Prepend velocity column to `D` and 'Q` dataframes
coeffs_dict['D'].insert(0, 'v', coeffs_dict['Fd']['v'])
coeffs_dict['Q'].insert(0, 'v', coeffs_dict['Fd']['v'])

# Split into center domain and boundary values
D_center, D_bdry = Dtools.split_domains(coeffs_dict['D'])
coeffs_dict['D Center'] = D_center
coeffs_dict['D Boundary'] = D_bdry

Q_center, Q_bdry = Dtools.split_domains(coeffs_dict['Q'])
coeffs_dict['Q Center'] = Q_center
coeffs_dict['Q Boundary'] = Q_bdry

### Distribution of field values

In [ ]:
fig, ax = plt.subplots()

marker_size = 0.05

coeffs_dict['D Center'].iloc[::5,:].plot('v', 'D0_y', kind='scatter', logx=False, logy=True, color='red', s=marker_size, ax=ax)
coeffs_dict['D Boundary'].iloc[::5,:].plot('v', 'D0_y', kind='scatter', logx=False, logy=True, color='blue', s=marker_size, ax=ax)

label_fontsize = 16
tick_size = 14

plt.xlabel(r'${\| v \|}_2$', fontsize=label_fontsize)
plt.ylabel(r'$\boldsymbol{D}_{xy}$', fontsize=label_fontsize)
ax.tick_params(axis='both', labelsize=tick_size)
plt.legend([r'$\boldsymbol{D}_{\Omega}$', r'$\boldsymbol{D}_{\partial \Omega}$'], fontsize=14, markerscale=20)
plt.grid()
# plt.savefig('../../langevin_study/figures/pre_ippl_meetings/2305/Dxy_maxwellian.png', dpi=200, bbox_inches="tight")

### Histogram over $v$

In [ ]:
coeffs_dict['D'].hist(column='v')

## Averaged Diffusion coefficient over each velocity bin

In [ ]:
Dnorm_avg_dict = {}
for key, D_df in coeffs_dict.items():
    Dnorm_avg_dict[key] = D_df.groupby('v').mean().reset_index()

In [ ]:
label_fontsize = 16
tick_size = 12
title_size = 17

logx = False
logy = False

sharex = True
sharey = True

fig, axes = plt.subplots(1,3, figsize=(12, 4), sharex=sharex, sharey=sharey)

matrix_el = 'D0_x'
y_axis_label = r'$\boldsymbol{D}_{xz}$'

plot_kind = 'scatter'

Dnorm_avg_dict['D Center'].plot('v', matrix_el, use_index=True, kind=plot_kind, logx=logx, logy=logy, color='blue', s=0.5, ax=axes[0])
axes[0].set_xlabel(r'${\| v \|}_2$', fontsize=label_fontsize)
axes[0].set_ylabel(y_axis_label, fontsize=label_fontsize)
axes[0].set_title(r'$\Omega$', fontsize=title_size)

Dnorm_avg_dict['D Boundary'].plot('v', matrix_el, use_index=True, kind=plot_kind, logx=logx, logy=logy, color='grey', s=0.5, ax=axes[1])
axes[1].set_xlabel(r'${\| v \|}_2$', fontsize=label_fontsize)
axes[1].set_ylabel(r'')
axes[1].set_title(r'$\partial \Omega$', fontsize=title_size)

Dnorm_avg_dict['D Center'].plot('v', matrix_el, use_index=True, kind=plot_kind, logx=logx, logy=logy, color='blue', s=0.5, ax=axes[2])
Dnorm_avg_dict['D Boundary'].plot('v', matrix_el, use_index=True, kind=plot_kind, logx=logx, logy=logy, color='grey', s=0.5, ax=axes[2])
axes[2].set_xlabel(r'${\| v \|}_2$', fontsize=label_fontsize)
axes[2].set_ylabel(r'')
axes[2].set_title(r'$\Omega \cup \partial \Omega$', fontsize=title_size)

for ax_el in axes:
    ax_el.legend().set_visible(False)
    ax_el.grid(True)
    ax_el.tick_params(axis='both', labelsize=tick_size)

plt.tight_layout()

# plt.savefig('../../langevin_study/figures/pre_ippl_meetings/2305/Dxz_avg_maxwellian.png', dpi=200, bbox_inches="tight")

# Plots for thesis

In [ ]:
figure_dir = 'D_figures'

## Distribution of $\boldsymbol D$

In [ ]:
# Load plotting style
sys.path.append('/home/tobiac/polybox/studium/mscThesis/personal_repo/data_visualization/src')
from mscPlotting import SizeParams
set_plotting_environment()

# Load convergence plot tools
from mscPlotting import prepare_asymptotic_plots

# Lineplot Kwargs specific to this section of the Notebook
lineplot_kwargs_dict = {
                   'logx' : False, 
                   'logy' : False, 
                   'color' : None}

## Cholesky Test of dump

In [ ]:
# Working on `coeffs_dict` previously loaded, containing all coefficients (Fd, D, Q)
coeffs_dict.keys()
coeffs_dict['D Center'].head()

In [ ]:
# Gather vectors and matrices as numpy arrays
Fd_vectors = Dtools.extract_coeffs(coeffs_dict['Fd'])
D_matrices = Dtools.extract_coeffs(coeffs_dict['D Center'].iloc[:,-9:])
Q_matrices = Dtools.extract_coeffs(coeffs_dict['Q Center'].iloc[:,-9:])

D_matrices.shape

In [ ]:
# Checks if matrices are positive-definite and symmetric (precondition for valid cholesky decomposition)
# Returns numpy and custom cholesky matrices of input matrices
# hessian_matrices.shape == (B,N,N)
def cholesky_test(hessian_matrices, Q_reference_matrices, error_limit=20):
    inp_shape = hessian_matrices.shape
    assert len(inp_shape) == 3 and inp_shape[1] == inp_shape[2]
    
    np_cholesky = []
    valid_indices = []
    matrices_not_pd = []
    matrices_not_semi_pd = []
    
    Q_relative_error = np.zeros_like(hessian_matrices[0])
    
    caught_errors = 0
    for idx, (D_matrix, Q_matrix) in enumerate(zip(hessian_matrices, Q_reference_matrices)):
        # Check for semi-positive-definitness of matrix
        e_vals, _ = np.linalg.eig(D_matrix)
        if e_vals.all() < 0.0:
            matrices_not_semi_pd.append(D_matrix)
            
        try:        
            # Compute Cholesky with two different methods
            Lt, Diag, _ = scipy.linalg.ldl(D_matrix)
            true_Q = np.sqrt(Diag)*Lt
            
            # true_Q = apply_LDLt_cholesky3x3(D_matrix)
            
            D_matrix = np.diag(np.diag(D_matrix))
            Lt, Diag, _ = scipy.linalg.ldl(D_matrix)
            true_Q = np.sqrt(Diag)*Lt
            
            np_cholesky.append(true_Q)

            valid_indices.append(idx)

        except Exception as err:
            if caught_errors < error_limit:
                print(f'> At idx={idx}: {err}.')
            else:
                np_cholesky = np.array(np_cholesky)
                valid_indices = np.array(valid_indices)
                matrices_not_pd = np.array(matrices_not_pd)
                return valid_indices, np_cholesky, matrices_not_pd
            caught_errors += 1

    np_cholesky = np.array(np_cholesky)
    valid_indices = np.array(valid_indices)
    matrices_not_pd = np.array(matrices_not_pd)

    # Check if Cholesky decompositions coincide
    total_error = np.zeros_like(hessian_matrices[0,...])
    cholesky_error_matrices = np.zeros_like(hessian_matrices)

    print(f'==================================================================')
    print(f'Encountered {caught_errors} errors when computing the Cholesky-Decomposition.')
    print(f'{len(matrices_not_semi_pd)}/{hessian_matrices.shape[0]} matrices are negative definite!')
    print(f'{np_cholesky.shape[0]}/{hessian_matrices.shape[0]} matrices are positive definite!')
    
    print('Error matrix per element:')
    print(total_error/np_cholesky.shape[0])

    return valid_indices, np_cholesky, matrices_not_pd, cholesky_error_matrices, total_error

In [ ]:
valid_indices, np_cholesky, matrices_not_pd = cholesky_test(D_matrices, Q_matrices)

### Some matrices are indeed negative definite

In [ ]:
e_vals, _ = np.linalg.eig(D_matrices[2])
e_vals

In [ ]:
Lt, Diag, _ = scipy.linalg.ldl(D_matrices[2])
print(f'Lt:\n{Lt}\nDiag:\n{Diag}')

### Our computation of Q requires the sqrt. Thus it fails for negative definite matrices

In [ ]:
true_Q = Dtools.apply_LDLt_cholesky3x3(D_matrices[2])

# Create encoded cube slice determining matrix type

In [ ]:
# Given cube of size [N^3, M, M] returns [N^2, M, M] at given idx along z axis
def create_cube_zslice(matrix_cube, idx=None):
    N3 = matrix_cube.shape[0]
    trailing_axes_size = matrix_cube.shape[1:]
    N = int((N3+1)**(1.0/3.0))

    if idx is None:
        idx = N//2

    return matrix_cube.reshape(N,N,N,*trailing_axes_size)[:,:,idx,...].reshape(N*N,-1), N

# Input shape: (N^3,M,M)
def create_encoded_slice(cube_field):
    cube_slice, M = create_cube_zslice(cube_field)
    
    # Create output matrix with scalars as fields (same size as `cube_slice`)
    output_encoding = np.zeros(cube_slice.shape[0], dtype=np.int8)
    
    # Loop through entries
    for idx in range(cube_slice.shape[0]):
        # Compute eigendecomposition
        e_vals, _ = np.linalg.eig(cube_slice[idx].reshape(3,3))
        
        # Assign label according to property
        if np.any(e_vals < 0): # negative
            if np.all(e_vals < 0):
                output_encoding[idx] = 0 # negative definite
            else:
                output_encoding[idx] = 1 # negative semi-definite
        else:
            if np.all(e_vals > 0):
                output_encoding[idx] = 3 # positive definite
            else:
                output_encoding[idx] = 2 # positive semi-definite
    
    return output_encoding.reshape(M,M)

def plot_encoded_slice(encoded_data, ax, plot_legend=False, vmax=5e7):
    
    # Create colormap containing a color for each possible value in the data
    colormap = ListedColormap(sns.color_palette("colorblind").as_hex(), N=4)
    cmap_list = [colormap(i) for i in range(colormap.N)]
    # Rotate colormap by one element (results in nicer color combinations for values {1,3})
    cmap_list = cmap_list[1:] + cmap_list[:1]

    labels = ['negative-definite', 'negative semi-definite', 'positive semi-definite', 'positive-definite']

    # Remove labels that are not present in data
    removed_els = 0
    for i in range(len(cmap_list)):
        if np.sum(encoded_data == i) == 0:
            del cmap_list[i - removed_els]
            del labels[i - removed_els]
            removed_els += 1

    # Generate new colormap only containing existing values
    cmap = ListedColormap(cmap_list)

    # Create a patch (proxy artist) for every color
    patches = [ mpatches.Patch(color=cmap_list[i], label=f'{labels[i]}') for i in range(len(cmap_list)) ]
    
    if plot_legend == True:
        # Put those patched as legend-handles into the legend
        ax.legend(handles=patches, loc='upper center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, fontsize=SizeParams().ticksize)

    
    ax.imshow(encoded_data, cmap=cmap, extent=[-vmax,vmax,-vmax,vmax])
    ax.grid()
    
    return patches

## Hessian over time

In [ ]:
Dfield_keys = ['0400', '0600', '0800', '0999']
file_pattern = 'langevin_Danalysis_0713_0833/Dfield_it'

num_axes = len(Dfield_keys)
fig, axes = plt.subplots(1, num_axes, figsize=(num_axes*5, 5), sharey=True)

coeffs_dict = {}

for (Dkey, ax) in zip(Dfield_keys, axes):
    # Load data, take center domain, extract matrix format entries, classify matrices with label
    coeffs_dict[Dkey] = Dtools.create_encoded_slice(Dtools.extract_coeffs(Dtools.split_domains(pd.read_csv(f'{file_pattern}{Dkey}.csv'))[0]))
    patches_for_label = Dtools.plot_encoded_slice(coeffs_dict[Dkey], ax)
    
    time_str = f'{Dkey.lstrip("0")}'
    ax.set_title(r'$t\ =\ $' + f'{Dkey.lstrip("0")}' + r'$dt$', y=1.05, fontsize=SizeParams().label_fontsize)
    
    x_label = r'$\boldsymbol v_x$'
    y_label = ''
    
fig.legend(handles=patches_for_label, loc='upper center', bbox_to_anchor=(0.5, -0.03), fancybox=True, shadow=True, fontsize=SizeParams().ticksize)

# Put ylabel on first plot only
y_label = r'$\boldsymbol v_y$'
axes[0].set_ylabel(y_label)

# fig.savefig(figure_dir+f'/D_cholesky_time.pdf', bbox_inches = "tight"),

## Hessian type analysis

In [ ]:
Dfield_keys = ['FDhessian_hockney/Dfield_it1000.csv',
              'FDhessian_vico/Dfield_it1000.csv',
              'spectralHessian_hockney/Dfield_it1000.csv',
              'spectralHessian_vico/Dfield_it1000.csv']

axes_titles = [r'FD $H_g$ (Hockney)',
              r'FD $H_g$ (Vico)',
              r'Spectral $H_g$ (Hockney)',
              r'Spectral $H_g$ (Vico)']

num_axes = len(Dfield_keys)
fig, axes = plt.subplots(1, num_axes, figsize=(num_axes*5, 5), sharey=True)

coeffs_dict = {}

for (Dkey, ax, ax_title) in zip(Dfield_keys, axes, axes_titles):
    coeffs_dict[Dkey] = Dtools.create_encoded_slice(Dtools.extract_coeffs(Dtools.split_domains(pd.read_csv(f'{Dkey}'))[0]))
    patches_for_label = Dtools.plot_encoded_slice(coeffs_dict[Dkey], ax)
    
    ax.set_title(ax_title, y=1.05, fontsize=SizeParams().label_fontsize)

fig.legend(handles=patches_for_label, loc='upper center', bbox_to_anchor=(0.5, 0.05), fancybox=True, shadow=True, fontsize=SizeParams().ticksize)

# fig.savefig(figure_dir+f'/hessian_comparison.pdf', bbox_inches = "tight"),

In [ ]:
test_matrices = Dtools.extract_coeffs(Dtools.split_domains(pd.read_csv(f'../../langevin_study/langevin_data/submission_data/langevin_P3M_Fd_D_r256_v64_vico_0712_1412/Dfield_it1199.csv'))[0])

test_matrix1 = test_matrices[0]
test_matrix2 = test_matrices[1]

print(f'1: {test_matrix1}\n')
print(f'2: {test_matrix2}\n')


# Set off-diagonals to zero:
# test_matrix[test_matrix < 1e-14] = 0.0

e_vals, _ = np.linalg.eig(test_matrix1)
print(f'Evals = {e_vals}\n')
print(f'{test_matrix1}\n')
print(f'{Dtools.apply_LDLt_cholesky3x3_safe(test_matrix1)}')

print(f'{Dtools.apply_LDLt_cholesky3x3_numpy(np.diag(np.diag(test_matrix1)))}')

In [ ]:
fig, ax = plt.subplots()

patches_for_label = Dtools.plot_encoded_slice(Dtools.create_encoded_slice(Dtools.extract_coeffs(Dtools.split_domains(pd.read_csv('langevin_Danalysis_0713_0833/Dfield_it0800.csv'))[0])), ax)

# patches_for_label = plot_encoded_slice(create_encoded_slice(extract_coeffs(split_domains(pd.read_csv(f'../../langevin_study/langevin_data/submission_data/langevin_P3M_Fd_D_r256_v64_vico_0712_1412/Dfield_it1199.csv'))[0])), ax)

fig.legend(handles=patches_for_label, loc='upper center', bbox_to_anchor=(0.5, 1.1), fancybox=True, shadow=True, fontsize=SizeParams().ticksize)

# fig.savefig('/home/tobiac/polybox/studium/mscThesis/personal_repo/presentations/langevin_discussions/0711/hessian_comparison.pdf', bbox_inches = "tight"),

x_label = r'$\boldsymbol v_x$'
y_label = r'$\boldsymbol v_y$'

prepare_imshow_plots(ax, x_label, y_label, legend_loc='right', bbox_to_anchor=None, ncol=1)

# fig.savefig(figure_dir+f'/D_cholesky_it0800.pdf', bbox_inches = "tight")

## Test that our LDLT decomposition works correctly

In [ ]:
# For any matrix A, A*A^T is positive semi-definite
def make_spd_matrix(dim=3):
    mat = np.random.rand(dim,dim)
    return np.dot(mat, mat.transpose())

# Create spd matrix that has a large condition number (|\lambda_{max}|/|\lambda_{min}| >> 0)
def make_hard_spd_matrix(dim=3, condition_number_magnitude=1e8):
    eigenvalues = np.diag(np.random.rand(dim))
    # Scale a random eigenvalue
    rand_idx = np.random.randint(1)
    eigenvalues[rand_idx, rand_idx] *= condition_number_magnitude
    
    eigenvalues = np.sqrt(eigenvalues)

    R = np.random.rand(dim,dim)

    return np.dot(np.dot(np.dot(R, eigenvalues), eigenvalues), R.transpose())

In [ ]:
# Generate random SPD matrices
N_spd_matrices = 10000

spd_matrices = np.array([make_hard_spd_matrix(3) for i in range(N_spd_matrices)])

error_limit = 10
incorrectly_decomposed = []

for idx, spd_matrix in enumerate(spd_matrices):
    caught_errors = 0
    try:
        Q = Dtools.apply_LDLt_cholesky3x3(spd_matrix)
        
    except Exception as err:
        incorrectly_decomposed.append(spd_matrix)
        if caught_errors < error_limit:
            print(f'> At idx={idx}: {err}.')
        caught_errors += 1

In [ ]:
len(incorrectly_decomposed)

In [ ]:
Q = Dtools.apply_LDLt_cholesky3x3(spd_matrices[4])

In [ ]:
e_vals, _ = np.linalg.eig(incorrectly_decomposed)
e_vals

In [ ]:
incorrectly_decomposed